<a href="https://colab.research.google.com/github/piyushjain4/Learn_langchain/blob/main/chapters/07-lcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### LangChain Essentials Course

# LangChains Expression Language

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Expression Langauge (LCEL), abstracting a full chain and understanding how it will work. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16 \
  langsmith==0.3.4 \
  docarray==0.40.0 \
  langchain-ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 400.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.3/333.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00


---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

---

> ⚠️ If using LangSmith, add your API key below:

In [2]:
from google.colab import userdata
Lang_api =userdata.get('Langsmith_api')

In [3]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGCHAIN_API_KEY"] =Lang_api

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-lcel-openai"

In [ ]:
!apt update && apt install -y pciutils lshw

In [16]:
! curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [17]:
!ollama serve > /dev/null 2>&1 &

In [18]:
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling ff82381e2bea... 100% ▕▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling 491dfa501e59... 100% ▕▏  801 B                         
pulling ed11eda7790d... 100% ▕▏   30 B                         
pulling 42347cd80dc8... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [19]:
import os
from langchain_ollama.chat_models import ChatOllama

model_name = "mistral"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)
creative_llm = ChatOllama(temperature=0.9, model=model_name)

---

## Traditional Chains vs LCEL

In this section we're going to dive into a basic example using the traditional method for building chains before jumping into LCEL. We will build a pipeline where the user must input a specific topic, and then the LLM will look and return a report on the specified topic. Generating a _research report_ for the user.

### Traditional LLMChain

The `LLMChain` is the simplest chain originally introduced in LangChain. This chain takes a prompt, feeds it into an LLM, and _optionally_ adds an output parsing step before returning the result.

Let's see how we construct this using the traditional method, for this we need:

* `prompt` — a `PromptTemplate` that will be used to generate the prompt for the LLM.
* `llm` — the LLM we will be using to generate the output.
* `output_parser` — an optional output parser that will be used to parse the structured output of the LLM.

In [8]:
from langchain import PromptTemplate

prompt_template = "Give me a small report on {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)

For the LLM, we'll start by initializing our connection to the OpenAI API. We do need an OpenAI API key, which you can get from the [OpenAI platform](https://platform.openai.com/api-keys).

We will use the `gpt-4o-mini` model with a `temperature` of `0.0`:

In [ ]:
# import os
# from getpass import getpass
# from langchain_openai import ChatOpenAI

# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
#     or getpass("Enter your OpenAI API key: ")

# llm = ChatOpenAI(
#     model_name="gpt-4o-mini",
#     temperature=0.0,
# )

In [11]:
llm_out = llm.invoke("Hello there")
llm_out

KeyboardInterrupt: 

Then we define our output parser, this will be used to parse the output of the LLM. In this case, we will use the `StrOutputParser` which will parse the `AIMessage` output from our LLM into a single string.

In [20]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [10]:
out = output_parser.invoke(llm_out)
out

NameError: name 'llm_out' is not defined

Through the `LLMChain` class we can place each of our components into a linear `chain`.

In [21]:
from langchain.chains import LLMChain

chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)

Note that the `LLMChain` _was_ deprecated in LangChain `0.1.17`, the expected way of constructing these chains today is through LCEL, which we'll cover in a moment.

We can `invoke` our `chain`, providing a `topic` that we'd like to be researched.

In [22]:
result = chain.invoke("retrieval augmented generation")
result

{'topic': 'retrieval augmented generation',
 'text': ' Title: Retrieval-Augmented Generation: A New Approach to Text Generation\n\nIntroduction:\nRetrieval-augmented generation (RAG) is an innovative approach in the field of text generation that combines the strengths of retrieval models and generative models. This hybrid method has shown promising results in various applications, such as answering questions, summarizing texts, and generating creative content.\n\nKey Components:\n1. Retrieval Models: These models are designed to search large-scale text corpora efficiently to find relevant information. They are particularly effective when the answer or the desired text snippet is not easily generated from scratch but can be found in an existing dataset.\n2. Generative Models: These models generate new text based on learned patterns and structures from the training data. They excel at creating coherent and contextually relevant sentences, even when the information they need is not readil

We can view a formatted version of this output using the `Markdown` display:

In [23]:
from IPython.display import display, Markdown

display(Markdown(result["text"]))

 Title: Retrieval-Augmented Generation: A New Approach to Text Generation

Introduction:
Retrieval-augmented generation (RAG) is an innovative approach in the field of text generation that combines the strengths of retrieval models and generative models. This hybrid method has shown promising results in various applications, such as answering questions, summarizing texts, and generating creative content.

Key Components:
1. Retrieval Models: These models are designed to search large-scale text corpora efficiently to find relevant information. They are particularly effective when the answer or the desired text snippet is not easily generated from scratch but can be found in an existing dataset.
2. Generative Models: These models generate new text based on learned patterns and structures from the training data. They excel at creating coherent and contextually relevant sentences, even when the information they need is not readily available in the training data.

Benefits of RAG:
1. Improved Accuracy: By combining the strengths of both retrieval and generation models, RAG can provide more accurate and informative responses compared to using either model alone.
2. Scalability: Retrieval-augmented generation allows models to access a vast amount of information without needing to memorize every detail, making it scalable to large datasets.
3. Flexibility: RAG can be applied to various text generation tasks, such as question answering, summarization, and creative writing, by simply adjusting the retrieval and generation components accordingly.

Challenges and Future Directions:
1. Quality of Retrieved Information: Ensuring that the information retrieved is relevant, accurate, and up-to-date remains a challenge for RAG systems.
2. Coherence and Consistency: Generating coherent and consistent text from both retrieved and generated parts can be difficult, as the two components may not always align seamlessly.
3. Efficiency: Retrieving information from large datasets can be computationally expensive, making it important to develop efficient indexing and search strategies for RAG systems.
4. Ethical Considerations: As with any AI system, there are ethical concerns related to the accuracy, fairness, and potential misuse of information retrieved and generated by RAG models.

Conclusion:
Retrieval-augmented generation represents a promising new approach to text generation that leverages the strengths of both retrieval and generative models. By addressing the challenges associated with this hybrid method, we can expect significant advancements in various applications of text generation, from answering questions to creating engaging content.

That is a simple `LLMChain` using the traditional LangChain method. Now let's move onto LCEL.

## LangChain Expression Language (LCEL)

**L**ang**C**hain **E**xpression **L**anguage (LCEL) is the recommended approach to building chains in LangChain. Having superceeded the traditional methods with `LLMChain`, etc. LCEL gives us a more flexible system for building chains. The pipe operator `|` is used by LCEL to _chain_ together components. Let's see how we'd construct an `LLMChain` using LCEL.

In [24]:
lcel_chain = prompt | llm | output_parser

We can `invoke` this chain in the same way as we did before:

In [25]:
result = lcel_chain.invoke("retrieval augmented generation")
result

' Title: Retrieval-Augmented Generation: A New Approach to Text Generation\n\nIntroduction:\nRetrieval-augmented generation (RAG) is an innovative approach in the field of text generation that combines the strengths of retrieval models and generative models. This hybrid method has shown promising results in various applications, such as answering questions, summarizing texts, and generating creative content.\n\nKey Components:\n1. Retrieval Models: These models are designed to search large-scale text corpora efficiently to find relevant information. They are particularly effective when the answer or the desired text snippet is not easily generated from scratch but can be found in an existing dataset.\n2. Generative Models: These models generate new text based on learned patterns and structures from the training data. They excel at creating coherent and contextually relevant sentences, even when the information they need is not readily available in the training data.\n\nBenefits of RAG:

The output format is slightly different, but the underlying functionality and content being output is the same. As before, we can view a formatted version of this output using the `Markdown` display:

In [26]:
display(Markdown(result))

 Title: Retrieval-Augmented Generation: A New Approach to Text Generation

Introduction:
Retrieval-augmented generation (RAG) is an innovative approach in the field of text generation that combines the strengths of retrieval models and generative models. This hybrid method has shown promising results in various applications, such as answering questions, summarizing texts, and generating creative content.

Key Components:
1. Retrieval Models: These models are designed to search large-scale text corpora efficiently to find relevant information. They are particularly effective when the answer or the desired text snippet is not easily generated from scratch but can be found in an existing dataset.
2. Generative Models: These models generate new text based on learned patterns and structures from the training data. They excel at creating coherent and contextually relevant sentences, even when the information they need is not readily available in the training data.

Benefits of RAG:
1. Improved Accuracy: By combining the strengths of both retrieval and generation models, RAG can provide more accurate and informative responses compared to using either model alone.
2. Scalability: Retrieval-augmented generation allows models to access a vast amount of information without needing to memorize every detail, making it scalable to large datasets.
3. Flexibility: RAG can be applied to various text generation tasks, such as question answering, summarization, and creative writing, by simply adjusting the retrieval and generation components accordingly.

Challenges and Future Directions:
1. Quality of Retrieved Information: Ensuring that the information retrieved is relevant, accurate, and up-to-date remains a challenge for RAG systems.
2. Coherence and Consistency: Generating coherent and consistent text from both retrieved and generated parts can be difficult, as the two components may not always align seamlessly.
3. Efficiency: Retrieving information from large datasets can be computationally expensive, making it important to develop efficient indexing and search strategies for RAG systems.
4. Ethical Considerations: As with any AI system, there are ethical concerns related to the accuracy, fairness, and potential misuse of information retrieved and generated by RAG models.

Conclusion:
Retrieval-augmented generation represents a promising new approach to text generation that leverages the strengths of both retrieval and generative models. By addressing the challenges associated with this hybrid method, we can expect significant advancements in various applications of text generation, from answering questions to creating engaging content.

### How Does the Pipe Operator Work?

Before moving onto other LCEL features, let's take a moment to understand what the pipe operator `|` is doing and _how_ it works.

Functionality wise, the pipe tells you that whatever the _left_ side outputs will be fed as input into the _right_ side. In the example of `prompt | llm | output_parser`, we see that `prompt` feeds into `llm` feeds into `output_parser`.

The pipe operator is a way of chaining together components, and is a way of saying that whatever the _left_ side outputs will be fed as input into the _right_ side.

Let's make a basic class named `Runnable` that will transform our a provided function into a _runnable_ class that we will then use with the pipe `|` operator.

In [27]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other.invoke(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)

With the `Runnable` class, we will be able wrap a function into the class, allowing us to then chain together multiple of these _runnable_ functions using the `__or__` method.

First, let's create a few functions that we'll chain together:

In [28]:
def add_five(x):
    return x+5

def sub_five(x):
    return x-5

def mul_five(x):
    return x*5

Now we wrap our functions with the `Runnable`:

In [29]:
add_five_runnable = Runnable(add_five)
sub_five_runnable = Runnable(sub_five)
mul_five_runnable = Runnable(mul_five)

Finally, we can chain these together using the `__or__` method from the `Runnable` class:

In [30]:
chain = (add_five_runnable).__or__(sub_five_runnable).__or__(mul_five_runnable)

chain.invoke(3)

15

So we can see that we're able to chain together our functions using `__or__`. The pipe `|` operator is simply a shortcut for the `__or__` method, so we can create the exact same chain like so:

In [31]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

chain.invoke(3)

15

## LCEL `RunnableLambda`

The `RunnableLambda` class is LangChain's built-in method for constructing a _runnable_ object from a function. That is, it does the same thing as the custom `Runnable` class we created earlier. Let's try it out with the same functions as before.

In [32]:
from langchain_core.runnables import RunnableLambda

add_five_runnable = RunnableLambda(add_five)
sub_five_runnable = RunnableLambda(sub_five)
mul_five_runnable = RunnableLambda(mul_five)

We chain these together again with the pipe `|` operator:

In [33]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

And call them using the `invoke` method:

In [34]:
chain.invoke(3)

15

Now we want to try something a little more testing, so this time we will generate a report, and we will try and edit that report using this functionallity.

In [35]:
prompt_str = "give me a small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

In [36]:
chain = prompt | llm | output_parser

In [37]:
result = chain.invoke("AI")
display(Markdown(result))

 Title: An Overview of Artificial Intelligence (AI)

Introduction:
Artificial Intelligence (AI) is a branch of computer science that aims to create intelligent machines capable of performing tasks that would normally require human intelligence. This field has been growing rapidly, with significant advancements in recent years due to the availability of large amounts of data and powerful computing resources.

Key Concepts:
1. Machine Learning (ML): A subset of AI that enables systems to automatically learn and improve from experience without being explicitly programmed.
2. Deep Learning (DL): A subset of ML that is based on artificial neural networks with representation learning capabilities, capable of learning complex patterns in large datasets.
3. Natural Language Processing (NLP): A field of AI that focuses on the interaction between computers and human language, enabling machines to understand, interpret, and generate human language.
4. Robotics: The branch of AI dealing with the design, construction, operation, and use of robots, which can be programmed to perform a variety of tasks autonomously.

Applications:
1. Autonomous Vehicles: AI is used in self-driving cars for navigation, object recognition, and decision-making.
2. Healthcare: AI is used in medical imaging analysis, drug discovery, and personalized medicine.
3. Finance: AI is used in fraud detection, algorithmic trading, and credit risk assessment.
4. Entertainment: AI is used in video games, virtual assistants, and movie recommendations.

Challenges and Ethical Considerations:
1. Bias: AI systems can perpetuate or even amplify existing biases if they are trained on biased data.
2. Privacy: The use of AI raises concerns about data privacy and security, as large amounts of personal information are often required for training AI models.
3. Job Displacement: There is a fear that AI could displace human workers in various industries, leading to job losses.
4. Autonomy and Control: As AI systems become more autonomous, there is a need to establish guidelines for their use and control to ensure they are used ethically and responsibly.

Conclusion:
AI has the potential to revolutionize many aspects of our lives, but it also presents significant challenges that must be addressed. It is crucial to continue researching and developing AI in a responsible and ethical manner, ensuring its benefits are maximized while minimizing potential negative impacts.

Here we are making two functions, `extract_fact` to pull out the main content of our text and `replace_word` that will replace AI with Skynet!

In [ ]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x

old_word = "AI"
new_word = "skynet"

def replace_word(x):
    return x.replace(old_word, new_word)

Lets wrap these functions and see what the output is!

In [ ]:
extract_fact_runnable = RunnableLambda(extract_fact)
replace_word_runnable = RunnableLambda(replace_word)

In [ ]:
chain = prompt | llm | output_parser | extract_fact_runnable | replace_word_runnable

In [ ]:
result = chain.invoke("retrieval augmented generation")
display(Markdown(result))

#### Introduction
Retrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.
#### Concept Overview
RAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:
1. **Retrieval Component**: This part of the system searches a large corpus of documents or knowledge bases to find relevant information based on a given query. It employs techniques such as vector embeddings and similarity search to identify the most pertinent documents.
2. **Generation Component**: Once relevant documents are retrieved, the generative model (often based on architectures like Transformers) synthesizes a coherent response by incorporating the retrieved information. This allows the model to produce more informed and contextually appropriate outputs.
#### Advantages
- **Enhanced Knowledge Access**: RAG allows models to leverage vast external datasets, which can significantly improve the quality of responses, especially in domains requiring up-to-date or specialized knowledge.
- **Reduced Hallucination**: Traditional generative models sometimes produce inaccurate or fabricated information (a phenomenon known as "hallucination"). By grounding responses in retrieved documents, RAG can mitigate this issue.
- **Dynamic Adaptability**: The retrieval component can be updated independently of the generative model, allowing the system to adapt to new information without retraining the entire model.
#### Applications
RAG has a wide range of applications, including:
- **Question Answering**: Providing accurate answers to user queries by retrieving relevant documents and generating responses based on that information.
- **Chatbots and Virtual Assistants**: Enhancing conversational agents with the ability to pull in real-time data and provide contextually relevant answers.
- **Content Creation**: Assisting in generating articles, reports, or summaries by retrieving and synthesizing information from multiple sources.
#### Challenges
Despite its advantages, RAG faces several challenges:
- **Complexity**: The integration of retrieval and generation components can complicate the system architecture and increase computational requirements.
- **Quality of Retrieved Information**: The effectiveness of RAG heavily depends on the quality and relevance of the retrieved documents. Poor retrieval can lead to suboptimal generation.
- **Latency**: The retrieval process can introduce delays, which may affect user experience in real-time applications.
#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of natural language processing, combining the strengths of retrieval and generation to produce more accurate and contextually relevant outputs. As research and development in this area continue, RAG is poised to play a crucial role in various applications, enhancing the capabilities of skynet systems in understanding and generating human-like text. Future work will likely focus on improving retrieval efficiency, enhancing the quality of generated content, and addressing the challenges associated with system complexity and latency.

Those are our `RunnableLambda` functions. It's worth noting that all inputs to these functions are expected to be a SINGLE arguments. If you have a function that accepts multiple arguments, you can input a dictionary with keys, then unpack them inside the function.

## LCEL `RunnableParallel` and `RunnablePassthrough`

LCEL provides us with various `Runnable` classes that allow us to control the flow of data and execution order through our chains. Two of these are `RunnableParallel` and `RunnablePassthrough`.

* `RunnableParallel` — allows us to run multiple `Runnable` instances in parallel. Acting almost as a Y-fork in the chain.

* `RunnablePassthrough` — allows us to pass through a variable to the next `Runnable` without modification.

To see these runnables in action, we will create two data sources, each source provides specific information but to answer the question we will need both to fed to the LLM.

In [39]:
from langchain.embeddings import MistralEmbeddings
embeddings = MistralEmbeddings(model="mistral-embed")
from langchain.vectorstores import DocArrayInMemorySearch

embedding = OpenAIEmbeddings()

vecstore_a = DocArrayInMemorySearch.from_texts(
    [
        "half the info is here",
        "DeepSeek-V3 was released in December 2024"
    ],
    embedding=embedding
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    [
        "the other half of the info is here",
        "the DeepSeek-V3 LLM is a mixture of experts model with 671B parameters"
    ],
    embedding=embedding
)

ImportError: cannot import name 'MistralEmbeddings' from 'langchain.embeddings' (/usr/local/lib/python3.11/dist-packages/langchain/embeddings/__init__.py)

Here you can see the prompt does have three inputs, two for context and one for the question itself.

In [ ]:
prompt_str = """Using the context provided, answer the user's question.
Context:
{context_a}
{context_b}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt_str),
    HumanMessagePromptTemplate.from_template("{question}")
])

Here we are wrapping our vector stores as retrievers so they can be fitted into one big retrieval variable to be used by the prompt.

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b, "question": RunnablePassthrough()
    }
)

The chain we'll be constructing will look something like this:

![](https://github.com/aurelio-labs/langchain-course/blob/main/assets/lcel-flow.png?raw=1)

In [ ]:
chain = retrieval | prompt | llm | output_parser

We `invoke` it as usual.

In [ ]:
result = chain.invoke(
    "what architecture does the model DeepSeek released in december use?"
)
result

'The DeepSeek-V3 model, released in December 2024, uses a mixture of experts architecture with 671 billion parameters.'

With that we've seen how we can use `RunnableParallel` and `RunnablePassthrough` to control the flow of data and execution order through our chains.

---